In [43]:
import pandas as pd, numpy as np, os
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import AsinhStretch, AsymmetricPercentileInterval
%matplotlib inline

# Training data - Illustris

***


**Idea:**

1. Get Illustris *broadband* images for haloes of different stellar mass
2. Create a segmap using a **magnitude threshold reasonable for LSST** (e.g. 26 mag/arcsec2)
    * Illustris data is of similar resolution (per arcsec) to SDSS; but it works for HST as well if the Illustris galaxies were closer by. I could "move" the galaxies closer leading to higher resolution & SB threshold, but I don't need to I think - the training model should be able to account for this
    
3. Create a **field** from a range of galaxies, similar to [GZ images here](https://www.illustris-project.org/galaxy_obs/)
    * Projected to different redshifts; different stellar masses
    * Can impose galaxies on top of each other to create "merger-like" images
    * Add stars
4. Convolve to different seeing & image depth

***

**Sample**

1. Get Illustris [subhalo catalog](https://www.illustris-project.org/data/downloads/Illustris-1/) containing all subhaloes
2. Find all *field subhaloes* that do not have another galaxy nearby
    * Select those that belong to a group with only one subhalo with $M_\star > 10^{10} M_\odot$


## Downloading data

In [24]:
sample  = pd.read_csv('sample.csv')
api_key = "7f3fb8b4bec79b80b0186a83d75b1602"
path    = "data/raw"

In [25]:
def download_file(idx):
    idx      = int(idx)
    baseurl  = "http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/"
    url      = f"{baseurl}/{idx}/stellar_mocks/broadband.fits"
    cmd      = f"wget {url} -nc -O {path}/{idx}.fits --header=api-key:{api_key}"
    return os.system(cmd)

In [ ]:
for idx, row in sample.iterrows():
    print(idx, end=" ")
    download_file(int(row.id))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [37]:
files = ! ls {path}
print(len(files), "/", len(sample), "galaxies")

3062 / 3062 galaxies


## Broadband image format

FITS file with url `http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/XXX/stellar_mocks/broadband.fits` where `XXX` is subhalo ID

18 extensions:

0: Header (empty); shows meta parameters of the file <br>
1: `BROADBAND`: Table (empty); shows metadata<br>
2-5: `CAMERAN-PARAMETERS`: Camera parameters (Camera0, Camera1, Camera2, Camera3) 
   * `FOV`: field of view in radians
   * `cmeradist`: distance from origin to the camera in kpc
   * Data array contains psf...? Or something else
   
6: `GADGET` table - simulation parameters <br>
7: `INTEGRATED_QUANTITIES` table - integrated SED stuff
   * `L_bol_grid`: total bolometric luminosity [W]
   * `L_NSX`: bolometric luminosity in camera `X`
   
8: `MAKEGRID`: data about the grid, table <br>
9: `MCRX`: some other data about how the images were created
   * `camerafov`, `cameradistance` in kpc; `N_CAMERA`: number of cameras
   
10: `SCATTERING_LAMBDAS`: number of rays shot at different wavelengths, table <br>
11: `SFRHIST`: star formation history table <br>
12: `STELLARMODEL`: IMF parameters, table <br>
13: `FILTERS`: Different filtes, table <br>
14-17: `CAMERA0-BROADBAND-NONSCATTER`: integrated image over broadband filters from camera 0
   * Image shape: 256 x 256 x 36 for 36 filters
   * Units: W/m/m^2/sr
   * `UNITCONV`: internal units to surface brightness
   
   
***

32 Available filters; train on Cousins I for now

In [41]:
nfilt = 14

In [45]:
example  = True
test_id  = int(sample.iloc[1000].id)

if example:
    file = fits.open(f"{path}/{test_id}.fits")
    filt  = filts.loc[nfilt]
    print(filt["filter"])
    fig, axs = plt.subplots(1, 4, figsize=(16, 4))
    for i, ax in enumerate(axs):
        img = file[f"CAMERA{i}-BROADBAND-NONSCATTER"].data[nfilt, :, :]
        ax.imshow(stretch(norm(img)))
        ax.axis('off'); ax.set_title(f"Camera {i}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/422798.fits'

In [38]:
sample

,id,mstar
0,229474,44.439700
1,244157,35.484930
2,251100,38.566820
3,251546,37.880745
4,253907,22.230915
...,...,...
3057,525516,1.030647
3058,526649,1.084849
3059,532127,1.008326
3060,534166,1.025330
